**@author Güise Rodríguez**

**Execution date: 06/04/2019 21:30 **

# Instalo las librerías necesarias

In [0]:
!pip3 install requests
!pip3 install beautifulsoup4

# Importo las librerías necesarias

In [0]:
import requests, sys, re, csv
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

# It's show time!

## Funciones auxiliares

In [0]:
def get_author(extract):
  authors = []
  for autora in extract.find_all("a"):
    authors.append(re.sub(' +',' ',autora['title']))
  if authors:
    return ",".join(map(str,authors))
  #Si es un colectivo, el nombre aparecería con dicho formato
  return re.sub(' +',' ',re.sub('-','',re.sub('\n','',datos.contents[0]))).strip()

In [0]:
def get_data(extract):
    extra = extract.find("span",{"class": "location"})
    if 'lst-item-opinion' in extract.get("class"):
      type = "opinion"
      if extra:
        authorInfo = extra.getText()
        location = ""
    else:
      type = "news"
      if extra:
        authorInfo = ""
        location = extra.getText()
    if extra is None:
      authorInfo = ""
      location = ""
    return type, authorInfo, location

## Código principal

In [0]:
base = "https://www.eldiario.es/focos/"
focos = ['mejores_ciudades','vida_digital','creacion_cultural','medio_ambiente','maltrato_animal']
depth = [12,23,41,36,10]
header = ['headline','type','date','section','author','authorInfo','location','url']
csvData = []

In [0]:
for i, foco in enumerate(focos):
  for nPage in np.arange(depth[i])+1:
    target = base + foco + "/?page=" + str(nPage) 
    request = requests.get(target)
    if request.status_code != 200:
      sys.exit("Error Code: " + str(request.status_code))
    soup = BeautifulSoup(request.content)
    body = soup.find("div",{"class": "pg-body"})
    for noticia in body.find_all("li",{"class": "lst-item"}):
      type, authorInfo, location = get_data(noticia)
      datos = noticia.find("span",{"class": "byline"})
      headline = re.sub("\\\\'","\'",noticia.find("a")['title']) #Sustituyo "\\'" por una comilla simple
      date = datos.find("span",{"class": "date"}).getText()
      author = get_author(datos)
      url = "eldiario.es" + noticia.find("a")['href']
      csvData.append([headline,type,date,foco,author,authorInfo,location,url])

In [0]:
dfObj = pd.DataFrame(csvData, columns= header)
dfObj.to_csv("eldiario_news.csv",sep=";",index=False,quoting=csv.QUOTE_MINIMAL)